### 데이터 분석 연습하기
1. 고객 3500명에 대한 학습용 데이터를 이용하여 성별 예측 모형을 만든 후 <br>
이를 평가용 데이터에 적용하여 얻은 2482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하라<br>
제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

### 1.1데이터 탐색하기

데이터 가져오기

In [1]:
import pandas as pd

In [2]:
#주어진 데이터 파일을 모두 읽어서 데이터 프레임 변수에 저장하기
x_train = pd.read_csv('x_train.csv', encoding = 'CP949')
x_test = pd.read_csv('x_test.csv', encoding = 'CP949')
y_train = pd.read_csv('y_train.csv', encoding = 'CP949')

In [3]:
# x_train, x_test, y_train 상위 5개 값 확인하기
print(x_train.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  


In [4]:
print(x_test.head())

   cust_id       총구매액     최대구매액        환불금액 주구매상품 주구매지점  내점일수    내점당구매건수  \
0     3500   70900400  22000000   4050000.0    골프  부산본점    13   1.461538   
1     3501  310533100  38558000  48034700.0   농산물   잠실점    90   2.433333   
2     3502  305264140  14825000  30521000.0  가공식품  본  점   101  14.623762   
3     3503    7594080   5225000         NaN  주방용품  부산본점     5   2.000000   
4     3504    1795790   1411200         NaN   수산품  청량리점     3   2.666667   

     주말방문비율  구매주기  
0  0.789474    26  
1  0.369863     3  
2  0.083277     3  
3  0.000000    47  
4  0.125000     8  


In [5]:
print(y_train.head())

   cust_id  gender
0        0       0
1        1       0
2        2       1
3        3       1
4        4       0


행/열 확인하기

In [6]:
# 각 데이터 세트의 행과 열의 개수를 확인하기
print(x_train.shape) # 10개 칼럼(열)로 구성된 3500건(행)의 데이터

(3500, 10)


In [7]:
print(x_test.shape) # 10개 칼럼으로 구성된 2482건 데이터

(2482, 10)


In [8]:
print(y_train.shape) # 2개 칼럼으로 구성된 3500건의 데이터

(3500, 2)


요약정보 확인하기

In [9]:
# x_train 세트의 요약정보 확인하기
print(x_train.info()) # 학습데이터와 테스트 데이터의 결측치 결과가 동일하므로 학습데이터 기준으로 설명
# 환불금액 칼럼은 1205건으로 결측치 존재, '주구매상품','주구매지점' 칼럼은 object 타입으로 숫자형 데이터가 아님 범주형

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB
None


기초 통계량 확인하기

In [10]:
# x_train의 기초 통계량을 확인하고 가독성을 위해 행/열 바꿔서 출력
print(x_train.describe().T)
# 칼럼범위(mean)이 10^7 수준의 차이로 스케일링 필요 '주말방문비율' 평균은 약 0.3이고 '총구매액'은 91,919,250 이기에
# 한 데이터가 왜곡되지 않도록 범위를 동일한 범위로 변환해야함

          count          mean           std         min           25%  \
cust_id  3500.0  1.749500e+03  1.010507e+03         0.0  8.747500e+02   
총구매액     3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액    3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
환불금액     1205.0  2.407822e+07  4.746453e+07      5600.0  2.259000e+06   
내점일수     3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수  3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율   3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기     3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   

                  50%           75%           max  
cust_id  1.749500e+03  2.624250e+03  3.499000e+03  
총구매액     2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액    9.837000e+06  2.296250e+07  7.066290e+08  
환불금액     7.392000e+06  2.412000e+07  5.637530e+08  
내점일수     8.000000e+00  2.500000e+01  2.850000e+02  
내점당구매건수  2.333333e+00  3.37500

### 1.2 전처리하기 

불필요한 칼럼 삭제하기<br>
cust_id(고객ID)는 종속변수인 성별을 예측하는 정보가 아니고 각 행 데이터를 유일하게 구별하는 키 역할<br>
즉 동일한 cust_id 행은 없음 따라서 성별 값을 예측하는 데이터 분석 과정에서 불 필요하므로 삭제처리<br>
단 추후 테스트 데이터의 cust_id 칼럼 값은 최종 제출하는 결과 파일에 포함되어야 하므로 x_test_cust_id 변수를 별도로 저장 관리

In [11]:
# 테스트 데이터의 cust_id 값은 x_test_cust_id 변수에 저장
x_test_cust_id = x_test['cust_id']

In [12]:
# cust_id 칼럼 삭제하기
x_train=x_train.drop(columns = ['cust_id'])
x_test=x_test.drop(columns = ['cust_id'])
y_train=y_train.drop(columns = ['cust_id'])

In [13]:
# 칼럼이 삭제된 상위 5개 행 확인
print(x_train.head())

       총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수    주말방문비율  구매주기
0  68282840  11264000  6860000.0      기타   강남점    19  3.894737  0.527027    17
1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000  0.000000     1
2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000  0.000000     1
3  16077620   4935000        NaN      기타   광주점    18  2.444444  0.318182    16
4  29050000  24000000        NaN      보석  본  점     2  1.500000  0.000000    85


In [14]:
print(y_train.head())

   gender
0       0
1       0
2       1
3       1
4       0


결측치 처리하기

In [15]:
print(x_train.isnull().sum())

총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [16]:
# 환불금액 칼럼의 결측치는 0으로 대치하기
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)

In [17]:
# 결측치가 조치되었는지 '환불금액' 칼럼의 결측치 개수 확인하기
print(x_train['환불금액'].isnull().sum())

0


In [18]:
print(x_test['환불금액'].isnull().sum())

0


범주형 변수를 인코딩하기<br>
주구매상품, 주구매지점 칼럼에 대해 인코딩하기

In [19]:
# 주구매상품 칼럼에서 중복을 제외한 값들을 확인하기
print(x_train['주구매상품'].unique())

['기타' '스포츠' '남성 캐주얼' '보석' '디자이너' '시티웨어' '명품' '농산물' '화장품' '골프' '구두' '가공식품'
 '수산품' '아동' '차/커피' '캐주얼' '섬유잡화' '육류' '축산가공' '젓갈/반찬' '액세서리' '피혁잡화' '일용잡화'
 '주방가전' '주방용품' '건강식품' '가구' '주류' '모피/피혁' '남성 트랜디' '셔츠' '남성정장' '생활잡화'
 '트래디셔널' '란제리/내의' '커리어' '침구/수예' '대형가전' '통신/컴퓨터' '식기' '소형가전' '악기']


In [20]:
# 주구매상품 칼럼에서 중복을 제외한 값들의 개수세기
print(x_train['주구매상품'].unique().size)

42


In [21]:
# 주구매지점 칼럼에서 중복을 제외한 값들 확인
print(x_train['주구매지점'].unique())

['강남점' '잠실점' '관악점' '광주점' '본  점' '일산점' '대전점' '부산본점' '분당점' '영등포점' '미아점'
 '청량리점' '안양점' '부평점' '동래점' '포항점' '노원점' '창원점' '센텀시티점' '인천점' '대구점' '전주점'
 '울산점' '상인점']


In [22]:
#주구매지점 칼럼에서 중복을 제외한 값들의 개수 세기
print(x_train['주구매지점'].unique().size) # 이러한 수십종류 값을 인코딩 하는 경우 라벨 인코딩이 효율적

24


In [23]:
# sklearn 패키지의 preprocessing 모듈에서 LabelEncoder 함수 가져오기
from sklearn.preprocessing import LabelEncoder

In [24]:
# 라벨 인코딩 수행을 위한 encoder 함수 만들기
encoder = LabelEncoder()

In [25]:
# 주구매상품에 대해 라벨 인코딩을 수행한 후 주구매상품 칼럼으로 다시 저장하기
x_train['주구매상품']=encoder.fit_transform(x_train['주구매상품'])

In [26]:
# 라벨 인코딩 결과를 확인하기 위해 상위 10개 값 확인
print(x_train['주구매상품'].head(10)) # 라벨 인코딩의 변환 결과는 classes_ 키워드를 통해 확인 가능

0     5
1    21
2     6
3     5
4    15
5    11
6    22
7    13
8     5
9     9
Name: 주구매상품, dtype: int32


In [27]:
# 주구매상품 칼럼에 대한 라벨 인코딩의 변환 순서 확인하기
print(encoder.classes_) # 가공식품이면 0 가구면 1 등등등

['가공식품' '가구' '건강식품' '골프' '구두' '기타' '남성 캐주얼' '남성 트랜디' '남성정장' '농산물' '대형가전'
 '디자이너' '란제리/내의' '명품' '모피/피혁' '보석' '생활잡화' '섬유잡화' '셔츠' '소형가전' '수산품' '스포츠'
 '시티웨어' '식기' '아동' '악기' '액세서리' '육류' '일용잡화' '젓갈/반찬' '주류' '주방가전' '주방용품'
 '차/커피' '축산가공' '침구/수예' '캐주얼' '커리어' '통신/컴퓨터' '트래디셔널' '피혁잡화' '화장품']


In [28]:
# 테스트 데이터도 라벨 인코딩 수행하기
x_test['주구매상품']=encoder.fit_transform(x_test['주구매상품'])

In [29]:
# 주구매지점에 대해 라벨 인코딩을 수행한 후 주구매지점 칼럼으로 다시 저장하기
x_train['주구매지점']=encoder.fit_transform(x_train['주구매지점'])

In [30]:
print(x_train['주구매지점'])

0        0
1       19
2        1
3        2
4        8
        ..
3495     8
3496     9
3497    21
3498     8
3499     8
Name: 주구매지점, Length: 3500, dtype: int32


In [31]:
# 주구매지점 칼럼에 대한 라벨 인코딩의 변환 순서 확인하기
print(encoder.classes_)

['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']


In [32]:
# 테스트 데이터의 주구매지점 칼럼도 라벨 인코딩하기
x_test['주구매지점']=encoder.fit_transform(x_test['주구매지점'])

파생변수 만들기<br>
이전 결측치 처리를 한 환불금액 칼럼에 대해 파생변수 만들기<br>
환불금액이 0보다 크면 1 0과 같으면 0으로 변경<br>
이때 환불금액 칼럼에 대한 비교 조건문은 condition 변수에 저장

In [33]:
# 환불금액 칼럼이 0보다 큰지에 대한 조건을 condition 변수에 저장학
condition =x_train['환불금액']>0

In [34]:
# condition 조건에 맞으면 '환불금액_new' 칼럼을 1로 설정하기
x_train.loc[condition,'환불금액_new']=1

In [35]:
#condition 조건이 맞지 않으면 '환불금액_new' 칼럼을 0으로 설정
x_train.loc[~condition,'환불금액_new']=0

In [36]:
# 환불금액, 환불금액_new 칼럼을 비교하기
print(x_train[['환불금액','환불금액_new']])

           환불금액  환불금액_new
0     6860000.0       1.0
1      300000.0       1.0
2           0.0       0.0
3           0.0       0.0
4           0.0       0.0
...         ...       ...
3495        0.0       0.0
3496  6049600.0       1.0
3497        0.0       0.0
3498        0.0       0.0
3499  5973000.0       1.0

[3500 rows x 2 columns]


In [37]:
# 환불금액 칼럼의 삭제 내용은 바로 데이터 세트에 반영하기
x_train = x_train.drop(columns=['환불금액'])

In [38]:
# 테스트 데이터 세트에서도 '환불금액_new' 라는 파생변수 만들기 (~는 not 의미)
x_test.loc[condition,'환불금액_new']=1
x_test.loc[~condition,'환불금액_new']=0

In [39]:
# 환불금액 칼럼의 삭제 내용은 바로 데이터 세트에 반영하기
x_test=x_test.drop(columns=['환불금액'])

표준화 크기로 변환하기<br>

In [40]:
# 크기변환 전 x_train 세트의 기초 통계량 확인하기
print(x_train.describe().T)

           count          mean           std         min           25%  \
총구매액      3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액     3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
주구매상품     3500.0  1.461200e+01  1.301995e+01         0.0  5.000000e+00   
주구매지점     3500.0  1.073429e+01  5.636480e+00         0.0  8.000000e+00   
내점일수      3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수   3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율    3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기      3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   
환불금액_new  3500.0  3.442857e-01  4.752027e-01         0.0  0.000000e+00   

                   50%           75%           max  
총구매액      2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액     9.837000e+06  2.296250e+07  7.066290e+08  
주구매상품     9.000000e+00  2.200000e+01  4.100000e+01  
주구매지점     9.000000e+00  1.500000e+01  2.300000e

In [41]:
# sklearn 패키지의 preprocessing 모듈에서 StandardScaler 함수를 가져오기
from sklearn.preprocessing import StandardScaler

In [42]:
# 표준화 크기변환을 수행하기 위한 scaler 객체 만들기
scaler= StandardScaler()

In [43]:
# scaler 객체로 표준화 변환을 수행하고 x_train 칼럼명을 사용하기
x_train= pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)

In [44]:
# 테스트 데이터를 표준화 크기로 변환하기
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns=x_test.columns)

In [45]:
# 크기 변환 후 x_train 기초 통계량 확인하기
print(x_train.describe().T) # 평균은 0 표준편차는 1에 근접하게 변화함

           count          mean       std       min       25%       50%  \
총구매액      3500.0 -4.409171e-17  1.000143 -0.882909 -0.533218 -0.389621   
최대구매액     3500.0 -4.838986e-17  1.000143 -0.708278 -0.524864 -0.307219   
주구매상품     3500.0 -6.236281e-17  1.000143 -1.122438 -0.738357 -0.431093   
주구매지점     3500.0 -8.333017e-17  1.000143 -1.904703 -0.485175 -0.307733   
내점일수      3500.0  2.518303e-16  1.000143 -0.671807 -0.635003 -0.414180   
내점당구매건수   3500.0 -2.288170e-16  1.000143 -0.959661 -0.611003 -0.262346   
주말방문비율    3500.0  9.192647e-17  1.000143 -1.060530 -0.966329 -0.175472   
구매주기      3500.0 -1.220294e-16  1.000143 -0.846966 -0.685318 -0.321610   
환불금액_new  3500.0  3.445498e-16  1.000143 -0.724606 -0.724606 -0.724606   

               75%        max  
총구매액      0.089237  13.648260  
최대구매액     0.103110  21.475852  
주구매상품     0.567518   2.027026  
주구매지점     0.756913   2.176441  
내점일수      0.211486   9.780490  
내점당구매건수   0.282432  10.066639  
주말방문비율    0.489224   2.391196  
구매주

상관관계 확인하기<br>
독립변수 중에는 금액이라는 의미가 포함된 칼럼이 3개 있음<br>
총구매액, 최대구매액, 환불금액_new 독립변수간의 상관성을 확인하여 상관성이 높은 변수들 일부는 삭제한다<br>
결과 상 총구매액과 최대구매액간에 약 0.7 즉 70퍼의 상관관계를 확인할 수 있으므로 최대구매액 칼럼 삭제한다<br>
상관관계 계수의 절대값이 0.6 이상이면 강한 상관관계 의미

In [46]:
# 총구매액, 최대구매액, 환불금액_new 칼럼간의 상관관계 구하기
print(x_train[['총구매액','최대구매액','환불금액_new']].corr())

              총구매액     최대구매액  환불금액_new
총구매액      1.000000  0.700080  0.403357
최대구매액     0.700080  1.000000  0.330687
환불금액_new  0.403357  0.330687  1.000000


In [47]:
# x_train 세트에서 최대구매액 칼럼을 삭제한 후 x_train에 저장
x_train=x_train.drop(columns='최대구매액')

In [48]:
#x_test 세트에서 최대구매액 칼럼을 삭제한 후 x_test 에 저장
x_test = x_test.drop(columns='최대구매액')

### 1.3 학습하고 평가하기

데이터 학습시키기<br>
종속변수인 '성별' 칼럼을 분류하는 모델 만들기<br>
의사나무 분류기를 활용하여 데이터 학습한 모델 생성 후 predict() 함수로 추출된 결과는 데이터프레임 형태가 아니므로<br>
반드시 pd.DataFrame() 함수 사용하여 데이터 타입 변경

In [49]:
# sklearn 패키지의 tree 모듈에서 DecisionTreeClassifier 함수를 가져오기
from sklearn.tree import DecisionTreeClassifier

In [50]:
# 의사결정나무 방식으로 수행되는 model 객체 만들기
model = DecisionTreeClassifier()

In [51]:
# x_train,y_train 세트로 model 학습시키기
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [52]:
#학습된 model을 활용하여 테스트 데이터의 종속변수를 예측하기
y_test_predicted = model.predict(x_test)

In [53]:
#예측한 결과 출력하기
print(pd.DataFrame(y_test_predicted).head(3))
#정확한 성능을 위해 하이퍼 파라미터 튜닝하기

   0
0  1
1  0
2  1


하이퍼 파라미터 튜닝하기

In [54]:
# 괄호 안에 하이퍼 파라미터를 입력하여 모델 객체 생성하기
model=DecisionTreeClassifier(max_depth=10,criterion='entropy',random_state=10)

In [55]:
# x_train,y_train 세트로 model 학습시키기
model.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=10)

In [56]:
#학습된 model을 활용하여 테스트 데이터의 종속변수 예측하기
y_test_predicted=model.predict(x_test)

In [57]:
# 예측한 결과를 출력해보기
print(pd.DataFrame(y_test_predicted).head(3))

   0
0  1
1  0
2  1


GridSearchCV() 함수로 하이퍼 파라미터 튜닝하기<br>
교차검증과 더불어 하이퍼 파라미터의 최적 조합을 찾아줌

하이퍼 파라미터에서 최적의 조합을 찾기 위한 GridSearchCV() 함수와 병렬처리를 통해 빠른 결과를 얻기 위해<br>
Piepline()함수를 사용

In [58]:
# 하이퍼 파라미터 튜닝을 위한 GridSearchCV 함수 가져오기
from sklearn.model_selection import GridSearchCV

In [59]:
# 병렬처리를 위한 Pipeline 함수 가져오기
from sklearn.pipeline import Pipeline

파이썬 코드의 가독성을 목적으로 하이퍼 파라미터를 딕셔너리 형태로 정리<br>
해당 하이퍼파라미터들은 모델을 대상으로 공부시키는 경우에 활용할 것이므로 model__ 접두사 작성<br>
편의상 트리의 최대 깊이인 max_depth 파라미터와 트리 분기 기준인 criterion 파라미터로 튜닝을 수행

In [60]:
# 하이퍼 파라미터들과 튜닝할 값을 작성하기
parameters={'model__max_depth':[3,4,5,6],
           'model__criterion':['gini','entropy']}

표준화 크기변환과 의사결정나무 분류기를 동시에 병렬처리로 수행하도록 파이프라인 모델을 생성

In [61]:
#데이터 스케일링과 의사결정나무 분류 작업을 병렬처리하도록 모델 만들기
pipeline_model = Pipeline([('scaler',StandardScaler()),
                          ('model',DecisionTreeClassifier())])

GridSearchCV() 함수를 사용하여 생성한 pipeline_model 모델을 입력하고 조합할 파라미터 세트인 paramters 변수 주입<br>
교차 검증으로 모델의 성능을 올리기 위해 cv 옵션도 작성 다음 사례에서는 교차 검증을 3번 수행 (cv=3)

In [62]:
grid_model = GridSearchCV(pipeline_model,param_grid=
                         parameters,cv=3)

만들어진 grid_model 모델에 X_TRAIN, Y_TRAIN 변수를 넣어서 모델을 학습시킴<br>
모델 학습과 스케일링이 동시에 수행되므로 이전의 데이터 스케일링 작업을 굳이 별도로 수행할 필요 없음

In [63]:
print(grid_model.fit(x_train,y_train))

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', DecisionTreeClassifier())]),
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__max_depth': [3, 4, 5, 6]})


앞에서 학습시킨 grid_model 모델에서 최적의 하이퍼 파라미터로 만들어진 최상의 모델을 추출<br>
최상의 모델은 best_model 변수에 저장

In [64]:
best_model=grid_model.best_estimator_

만들어진 best_model 모델을 활용하여 x_test 세트에 대응되는 y_test 예측값을 계산<br>
계산된 y_test_predicted값은 다음을 통해 확인

In [65]:
y_test_predicted=best_model.predict(x_test)

In [66]:
print(pd.DataFrame(y_test_predicted))

      0
0     0
1     0
2     0
3     0
4     0
...  ..
2477  1
2478  0
2479  0
2480  0
2481  1

[2482 rows x 1 columns]


결과 예측하기 <br>
종속변수인 성별 값을 예측하기 위해서 predict() 함수를 사용했다면 <br>
이번 문제에서 요구한 성별에 대한 확률은 predict_proba() 함수를 사용함

In [67]:
# model을 통해 x_test에 맞는 종속변수 확률 구하기
y_test_proba = model.predict_proba(x_test)

In [68]:
#종속변수의 0,1에 대한 각 확률을 확인하기
print(pd.DataFrame(y_test_proba).head())

          0         1
0  0.200000  0.800000
1  1.000000  0.000000
2  0.000000  1.000000
3  0.548837  0.451163
4  1.000000  0.000000


최종적으로 구해야하는 확률은 남성이므로 [1] 코드로 남성 성별로 분류된 확률 확인<br>
확인 결과가 정상이라면 result 변수에 해당 결과를 저장

In [69]:
# 성별이 남성인 확률값을 데이터 프레임 형태로 출력하기
print(pd.DataFrame(y_test_proba)[1])

0       0.800000
1       0.000000
2       1.000000
3       0.451163
4       0.000000
          ...   
2477    0.462312
2478    0.000000
2479    0.338028
2480    1.000000
2481    1.000000
Name: 1, Length: 2482, dtype: float64


In [70]:
# 위 결과를 result 변수에 저장하기
result = pd.DataFrame(y_test_proba)[1]

모델 평가하기<br>
x_train, y_train으로 데이터를 학습시키고 학습된 모델을 가지고 테스트 데이터의 종속변수를 예측함<br>
예측된 y_test_predicted 값에 맞춰볼 수 있는 실제 y_test 값이 있다면 유의미한 평가 결과를 얻을 수 있지만<br>
현재 실제 종속변수 데이터 세트는 y_train 만 존재하므로 다음과 같은 코드를 작성한다

In [71]:
# x_train에 대한 종속변수를 예측하기
y_train_predicted = model.predict(x_train)

In [72]:
# roc 평가지표를 계산하기 위한 함수를 가져오기
from sklearn.metrics import roc_auc_score

In [73]:
# 학습 데이터에 대한 y_train, y_train_predicted 의 ROC 결과 확인하기
print(roc_auc_score(y_train,y_train_predicted))

0.6896432468240978


유의미한 학습모델 만들기

In [74]:
# 데이터 분리하기 위한 train_test_split 함수 가져오기
from sklearn.model_selection import train_test_split

In [76]:
# 학습용과 검증용을 8:2 로 분리하기
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train,y_train,\
                                                   test_size = 0.2,random_state=10)

In [77]:
# 분리된 데이터의 행/열 구조 확인하기
print(X_TRAIN.shape)

(2800, 8)


In [78]:
print(X_TEST.shape)

(700, 8)


In [79]:
print(Y_TRAIN.shape)

(2800, 1)


In [80]:
print(Y_TEST.shape)

(700, 1)


분리된 데이터로 의사결정나무 모델을 생성하고 모델을 평가하기

In [81]:
# 의사결정나무 모델을 생성하기
model= DecisionTreeClassifier(max_depth=10, criterion ='entropy',
                             random_state=10)

In [82]:
# 분리된 X_TRAIN,Y_TRAIN 으로 모델 학습시키기
model.fit(X_TRAIN,Y_TRAIN)

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=10)

In [84]:
# 학습한 모델을 통해 X_TEST 세트에 대응되는 Y_TEST 세트를 예측하기
Y_TEST_PREDICTED = model.predict(X_TEST)

학습이 완료된 이후에는 채점 기준인 roc_auc_score() 함수를 사용하여 평가하기

In [85]:
# sklearn 패키지의 metrics 모듈에서 roc_auc_score 함수 불러오기
from sklearn.metrics import roc_auc_score

In [86]:
#roc 평가지표 값 확인하기
print(roc_auc_score(Y_TEST,Y_TEST_PREDICTED))

0.5771332492643968


roc_auc_score 결과는 1에 가까울 수록 모델 성능이 우수한 것으로 시간적인 여유가 된다면 이 값을 높이도록 시도

### 1.4결과 제출하기
최종 데이터는 cust_id 값과 y_test의 예측값을 함께 출력해야한다<br>
concat() 함수를 사용하여 이전에 저장한 x_test_cust_id 변수와 result 변수를 세로 방향(열 기준)으로 통합

In [87]:
# x_test_cust_id 변수와 result 변수를 세로 방향으로 붙이기
pd.concat([x_test_cust_id,result],axis=1)

,cust_id,1
0,3500,0.800000
1,3501,0.000000
2,3502,1.000000
3,3503,0.451163
4,3504,0.000000
...,...,...
2477,5977,0.462312
2478,5978,0.000000
2479,5979,0.338028
2480,5980,1.000000


In [88]:
# '1'칼럼명을 'gender' 칼럼명으로 변환하여 다시 결과를 확인하기
pd.concat([x_test_cust_id,result],axis=1).rename(columns={1:'gender'})

,cust_id,gender
0,3500,0.800000
1,3501,0.000000
2,3502,1.000000
3,3503,0.451163
4,3504,0.000000
...,...,...
2477,5977,0.462312
2478,5978,0.000000
2479,5979,0.338028
2480,5980,1.000000


In [90]:
# 앞의 출력 결과를 final 변수에 저장하기
final = pd.concat([x_test_cust_id,result],axis=1).rename(columns={1:'gender'})

In [93]:
# final 변수를 data 디렉터리 하위에 12345.csv 이름으로 저장하기
final.to_csv('12345.csv',index=False)

제출된 파일의 정상 여부를 확인하기

In [92]:
final = pd.read_csv('12345.csv')
print(final)

      cust_id    gender
0        3500  0.800000
1        3501  0.000000
2        3502  1.000000
3        3503  0.451163
4        3504  0.000000
...       ...       ...
2477     5977  0.462312
2478     5978  0.000000
2479     5979  0.338028
2480     5980  1.000000
2481     5981  1.000000

[2482 rows x 2 columns]
